# Core Analysis: MARTA GTFS + Walk Isochrone

This notebook:
1. Downloads MARTA GTFS static data
2. Parses stops and identifies a sample downtown stop
3. Downloads Atlanta walking network using OSMnx
4. Creates a Pandana network for fast accessibility analysis
5. Computes a 500m walk isochrone from the sample stop
6. Exports GeoJSON files for visualization

**Author**: VIP Team 1270 - Smart Cities / Urban Systems  
**Project**: Atlanta SDG Portfolio Project


In [ ]:
# Core imports
import os
import sys
import zipfile
import requests
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import osmnx as ox
import pandana as pdna
import networkx as nx

print(f"OSMnx version: {ox.__version__}")
print(f"Pandana version: {pdna.__version__}")
print(f"GeoPandas version: {gpd.__version__}")


In [ ]:
# Set up paths - use relative paths from notebooks directory
BASE_DIR = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
if 'notebooks' in os.getcwd():
    os.chdir(BASE_DIR)
    
GTFS_DIR = os.path.join(BASE_DIR, 'gtfs_data')
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

# Create directories if they don't exist
for d in [GTFS_DIR, DATA_DIR, OUTPUT_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"GTFS directory: {GTFS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")


## 1. Download MARTA GTFS Data


In [ ]:
# MARTA GTFS URL (static feed)
# Source: https://itsmarta.com/app-developer-resources.aspx
MARTA_GTFS_URL = "https://itsmarta.com/MARTA-GTFS.zip"
GTFS_ZIP_PATH = os.path.join(GTFS_DIR, 'google_transit.zip')

def download_gtfs(url, output_path, force=False):
    """Download GTFS zip file if not already present."""
    if os.path.exists(output_path) and not force:
        print(f"GTFS file already exists: {output_path}")
        return True
    
    print(f"Downloading MARTA GTFS from {url}...")
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        with open(output_path, 'wb') as f:
            f.write(response.content)
        
        print(f"Downloaded successfully: {len(response.content) / 1024:.1f} KB")
        return True
    except Exception as e:
        print(f"Error downloading GTFS: {e}")
        return False

# Download GTFS
download_gtfs(MARTA_GTFS_URL, GTFS_ZIP_PATH)


In [ ]:
# Extract GTFS zip
def extract_gtfs(zip_path, extract_dir):
    """Extract GTFS zip file."""
    stops_path = os.path.join(extract_dir, 'stops.txt')
    if os.path.exists(stops_path):
        print(f"GTFS already extracted to {extract_dir}")
        return True
    
    print(f"Extracting GTFS zip...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted to {extract_dir}")
        return True
    except Exception as e:
        print(f"Error extracting GTFS: {e}")
        return False

extract_gtfs(GTFS_ZIP_PATH, GTFS_DIR)

# List extracted files
print("\nExtracted files:")
for f in os.listdir(GTFS_DIR):
    print(f"  - {f}")
